In [1]:
#import package
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

In [2]:
#load data
train=pd.read_csv("train.csv")
X=train["tweet"]
Y=train["class"]

In [3]:
#score
def score(pred_Y,test_Y):
    
    #AllF
    list_score_all=f1_score(pred_Y,test_Y,average=None)
    score_all=np.mean(list_score_all)
    
    #HateF
    h_pred_Y=pred_Y.copy()
    h_test_Y=test_Y.copy()
    h_pred_Y[h_pred_Y==2]=1
    h_test_Y[h_test_Y==2]=1
    list_score_hate=f1_score(h_pred_Y,h_test_Y,average=None)
    score_hate=np.mean(list_score_hate)
    
#     print("HateF:",score_hate)
#     print("AllF:",score_all)
#     print("Final:",0.6*score_hate+0.4*score_all)
    score_final=0.6*score_hate+0.4*score_all
    return score_final

In [4]:
#grid search
best_hyperparameters=[0]
list_depth=[4,5,6,7,8]
list_gamma=[0.05,0.1,0.3,0.5]
list_eta=[0.05,0.1,0.15,0.2,0.25,0.3]
for depth in list_depth:
    for gamma in list_gamma:
        for eta in list_eta:
            k=5
            n_samples=len(X)
            fold_size=n_samples//k
            scores=[]
            masks=[]
            for fold in range(k):

                #generate a boolean mask for the test set in this fold
                test_mask=np.zeros(n_samples,dtype=bool)
                test_mask[fold*fold_size:(fold+1)*fold_size]=True

                #create training and testing sets using this mask
                test_X,test_Y=X[test_mask],Y[test_mask]
                train_X,train_Y=X[~test_mask],Y[~test_mask]

                #TfidfVectorizer
                tfidf=TfidfVectorizer()
                tfidf.fit(train_X)
                train_X=tfidf.transform(train_X)
                test_X=tfidf.transform(test_X)

                #training
                clf=XGBClassifier(eval_metric='mlogloss',use_label_encoder=False,max_depth=depth,gamma=gamma,eta=eta)
                clf.fit(train_X,train_Y)
                pred_Y=clf.predict(test_X)
                scores.append(score(pred_Y,test_Y))
            
            if np.mean(scores)>best_hyperparameters[0]:
                best_hyperparameters=[np.mean(scores),depth,gamma,eta]
                
print(best_hyperparameters)
#score,depth,gamma,eta

[0.6649232053280147, 6, 0.1, 0.3]
